# 2. Route Points

> Takes `route_id` as input and returns geo-locations of the route to be taken in response. There isn't an identifier for the bus stop in this dataset. Instead it'points are at regular distance along the route.

## Sample request

```bash
curl -X POST 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/RoutePoints' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  -d '{"routeid":3796}'
```

## Sample response

```json
{
    "data": [
        {
            "latitude": "12.97751",
            "longitude": "77.57141",
            "responsecode": 200
        },
        {
            "latitude": "12.97749",
            "longitude": "77.57098",
            "responsecode": 200
        },
        {
            "latitude": "12.905319",
            "longitude": "77.543217",
            "responsecode": 200
        },
        {
            "latitude": "12.905319",
            "longitude": "77.543217",
            "responsecode": 200
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 560,
    "responsecode": 200
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.route_points

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm
import geojson

import requests
import pandas as pd
from fastcore.all import Path
from nbdev.config import get_config

from traffic_data_bengaluru.utils import *
from traffic_data_bengaluru.bmtc.apis.routes import get_routes

In [ ]:
#| export
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false
data_directory = get_data_directory() / "bmtc"

# Functions

In [ ]:
#| export
def fetch_route_points(route_id: str, sleep_duration: float = 0.1):
    """Fetches geo-locations for a given BMTC `route_id` from the API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/RoutePoints"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    payload = {"routeid": int(route_id)}
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
#| hide
#| eval: false

sample_route_id = 3796
route_points = fetch_route_points(route_id = sample_route_id)
print(json.dumps(route_points, indent=4))

In [ ]:
#| export
def convert_route_to_geojson(route, properties):
    """Converts route into a geojson Feature."""
    if route is None:
        return None
    coordinates = []
    for row in route['data']:
        # longitude is first, latitude comes next.
        coordinates.append([round(float(row['longitude']), 6), round(float(row['latitude']), 6)])
    geometry = geojson.LineString(coordinates)
    feature = geojson.Feature(geometry = geometry, properties = properties)
    return feature

In [ ]:
#| hide
#| eval: false
feature = convert_route_to_geojson(route_points, {"route_id": sample_route_id})
print(json.dumps(feature, indent=4))

In [ ]:
#| export
def get_route_id(filepath: Path):
    """The filepath has `route_id` at the end."""
    return filepath.name.split('.')[0]

In [ ]:
# | export
def task_fetch_route_points(data_directory: Path):
    logger.info("Fetching route points ...")

    df_routes = get_routes(data_directory)
    now = str(int(datetime.datetime.now().timestamp()))

    raw_directory = data_directory / 'raw' / 'route_points' / now
    raw_directory.mkdir(exist_ok=True, parents=True)

    for index, row in tqdm(df_routes.iterrows(), total = df_routes.shape[0], desc = 'Fetching route points'):
        trip_details = fetch_route_points(route_id = row['route_id']) 
        with open(raw_directory / f"{row['route_id']}.json", "w") as f:
            json.dump(trip_details, f, indent = 4)

    logger.info("Processing route points ...")
    cleaned_directory = data_directory / 'cleaned' / 'route_points' / now
    cleaned_directory.mkdir(exist_ok=True, parents=True)

    features = []
    for filepath in tqdm(raw_directory.ls(), total = raw_directory.ls().__len__(), desc = 'Converting route points to geojson'):
        route = read_file(filepath)
        if route is None:
            continue
        route_id = get_route_id(filepath)
        properties = {'route_id': route_id}
        
        feature = convert_route_to_geojson(route, properties)
        with open(cleaned_directory / f"{route_id}.json", "w") as f:
            json.dump(feature, f, indent = 4)
        features.append(feature)

    # Write to a file as a geojson FeatureCollection.
    fc = geojson.FeatureCollection(features = features)
    with open(data_directory / 'cleaned' / 'route_points' / f'{now}.geojson', 'w') as f:
        json.dump(fc, f)

In [ ]:
#| hide
#| eval: false

task_fetch_route_points(data_directory=data_directory)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()